In [2]:
import csv
from pathlib import Path
import requests
from bs4 import BeautifulSoup
from PIL import Image
import base64
import pandas as pd

In [3]:
url = "https://www.startech.com.bd/component/processor/intel-processor?sort=p.price&order=ASC&limit=90"
responce = requests.get(url)
html = responce.text
soup = BeautifulSoup(html, "lxml")

product_divs = soup.find_all("div", class_="p-item")

# List to hold all product data
products = []


In [ ]:

for product in product_divs:
    product_name = product.find("h4", class_="p-item-name").text.strip()
    product_image = product.find("img")["src"]
    
    # Download and save the product image
    image_data = requests.get(product_image).content
    Path(f'{product_name}.jpg').write_bytes(image_data)
    
    product_features = product.find_all("li")
    feature_list = [feature.text.strip() for feature in product_features]

    product_price = product.find("div", class_="p-item-price").text.strip()
    
    # Add product data to the list
    products.append({
        "name": product_name,
        "price": product_price,
        "features": feature_list,
        "image": f'{product_name}.jpg'  # Store the image filename
    })

# Define the CSV file columns
fieldnames = ["name", "price", "features", "image"]

# Write to a CSV file
with open('products.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for product in products:
        # Convert the features list to a single string separated by commas
        product["features"] = ', '.join(product["features"])
        writer.writerow(product)

print("Data has been written to products.csv")
